## Installing libraries

In [ ]:
!pip install "transformers==4.35" "datasets==2.13.0" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1" "tiktoken"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 16.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other 

In [ ]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
from datasets import Dataset, load_dataset
from random import randrange
from peft import LoraConfig, get_peft_model, AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer

# Load dataset from local

In [ ]:
df = pd.read_csv("/content/Manual.Q&A - 2022.csv")
# df = pd.read_csv("/content/qna_data.csv")

In [ ]:
df.head()

,Serial no,Question,Answer
0,NaN,NaN,NaN
1,1.0,추진 목적은 ?,국민들이 저렴한 보험료로 예기치 못한 풍수해에 스스로 대비하는 책임의식 강화 및 피...
2,2.0,근거법령은 ?,풍수해보험법
3,3.0,대상 재해는 무엇인가 ?,"태풍, 홍수, 호우, 강풍, 풍랑, 해일, 대설, 지진(지진해일 포함)"
4,4.0,보험 목적물은 ?,"주택, 농･임업용 온실(비닐하우스 포함), 상가･공장(소상공인)"


In [ ]:
df.drop(0,axis=0,inplace=True)

In [ ]:
# Concatenate 'Question' and 'Answer' columns into a new column 'text'
# df['text'] = '<s>' + '[Q]' + df['Question'] + '[/Q]' + '[A]' + df['Answer'] + '[/A]' + '</s>'
df['text'] = '질문:\n' + df['Question'] + '\n\n답변:\n' + df['Answer']

In [ ]:
df.head()

,Serial no,Question,Answer,text
1,1.0,추진 목적은 ?,국민들이 저렴한 보험료로 예기치 못한 풍수해에 스스로 대비하는 책임의식 강화 및 피...,질문:\n추진 목적은 ? \n\n답변:\n국민들이 저렴한 보험료로 예기치 못한 풍수...
2,2.0,근거법령은 ?,풍수해보험법,질문:\n근거법령은 ? \n\n답변:\n풍수해보험법
3,3.0,대상 재해는 무엇인가 ?,"태풍, 홍수, 호우, 강풍, 풍랑, 해일, 대설, 지진(지진해일 포함)","질문:\n대상 재해는 무엇인가 ? \n\n답변:\n태풍, 홍수, 호우, 강풍, 풍랑..."
4,4.0,보험 목적물은 ?,"주택, 농･임업용 온실(비닐하우스 포함), 상가･공장(소상공인)","질문:\n보험 목적물은 ? \n\n답변:\n 주택, 농･임업용 온실(비닐하우스 포함..."
5,5.0,보험료 지원은 ?,총 보험료의 70~87.04%* 를 정부에서 지원(최대 92%**)\r,질문:\n보험료 지원은 ? \n\n답변:\n총 보험료의 70~87.04%* 를 정부...


In [ ]:
print(df['text'].loc[1])

질문:
추진 목적은 ? 

답변:
국민들이 저렴한 보험료로 예기치 못한 풍수해에 스스로 대비하는 책임의식 강화 및 피해 발생시 복구에 필요한 현실적 보상을 통한 조기 생활안정 정착에 기여


In [ ]:
#drop columns other than 'text'
df.drop(columns=['Question','Answer'], axis=1, inplace=True)

In [ ]:
df.columns
df.drop(columns=['Serial no '], axis=1, inplace=True)

In [ ]:
df.head()

,text
1,질문:\n추진 목적은 ? \n\n답변:\n국민들이 저렴한 보험료로 예기치 못한 풍수...
2,질문:\n근거법령은 ? \n\n답변:\n풍수해보험법
3,"질문:\n대상 재해는 무엇인가 ? \n\n답변:\n태풍, 홍수, 호우, 강풍, 풍랑..."
4,"질문:\n보험 목적물은 ? \n\n답변:\n 주택, 농･임업용 온실(비닐하우스 포함..."
5,질문:\n보험료 지원은 ? \n\n답변:\n총 보험료의 70~87.04%* 를 정부...


In [ ]:
#convert to Huggingface Datasets format
train = Dataset.from_pandas(df)

In [ ]:
train

Dataset({
    features: ['text'],
    num_rows: 25
})

# Load the dataset from Huggingface

In [ ]:
# !huggingface-cli login

In [ ]:
# from datasets import load_dataset, Dataset
# dataset = load_dataset("HuggingFaceH4/no_robots")

In [ ]:
# dataset

# Fine-Tuning

In [ ]:
model_id = "davidkim205/komt-mistral-7b-v1"
# model_id = "genaitraining/llama-2-7b-domain-tuned"


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' cr

In [ ]:
# Get the type
compute_dtype = getattr(torch, "float16")

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
)


In [ ]:
%%time
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True,use_cache= False)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

CPU times: user 286 ms, sys: 24.4 ms, total: 311 ms
Wall time: 1.56 s


In [ ]:
%%time
# Load the pretrained model
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map="auto")

Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

CPU times: user 23.8 s, sys: 25.8 s, total: 49.6 s
Wall time: 2min 36s


In [ ]:
# LoRA config based on QLoRA paper
peft_config = LoraConfig(
                          lora_alpha=16,
                          lora_dropout=0.1,
                          r=64,
                          bias="none",
                          task_type="CAUSAL_LM"
                        )

In [ ]:
# Define the training arguments. For full list of arguments, check
#https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments
args = TrainingArguments(
    output_dir='llama2-7b-tuned-korean',
    num_train_epochs=1, # adjust based on the data size
    per_device_train_batch_size=2, # use 4 if you have more GPU RAM
    save_strategy="epoch", #steps
    # evaluation_strategy="epoch",
    learning_rate=2e-4,
    fp16=True,
    seed=42,
    gradient_checkpointing=False

)

In [ ]:
# Create the trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train,
    # eval_dataset=test,
    dataset_text_field='text',
    peft_config=peft_config,
    max_seq_length=512,
    tokenizer=tokenizer,
    args=args,
    packing=True
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


In [ ]:
# Training Params
train_params = TrainingArguments(
    output_dir="llama2-7b-tuned-korean",
    num_train_epochs=20,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=10,
    logging_steps=10,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant"
)

# Trainer
fine_tuning = SFTTrainer(
    model=model,
    train_dataset=train,
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=train_params
)

# Training
fine_tuning.train()
fine_tuning.save_model()

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/25 [00:00<?, ? examples/s]

You are using 8-bit optimizers with a version of `bitsandbytes` < 0.41.1. It is recommended to update your version as a major bug has been fixed in 8-bit optimizers.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,4.390900
20,2.974500
30,2.529000
40,1.925500
50,1.412200
60,0.914200
70,0.587400
80,0.391500
90,0.250300
100,0.187900


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

In [ ]:
# train
trainer.train()

In [ ]:
# save model in local
# trainer.save_model()
fine_tuning.save_model()

# Merge the base model and adapters and save it

Clean the memory

In [ ]:
# Empty VRAM
del model
del fine_tuning
import gc
gc.collect()
gc.collect()

0

In [ ]:
torch.cuda.empty_cache()

In [ ]:
gc.collect()

28

Reload the saved model and merge it then we can save the whole model

In [ ]:
%%time
from peft import AutoPeftModelForCausalLM

new_model = AutoPeftModelForCausalLM.from_pretrained(
    'llama2-7b-tuned-korean',
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",offload_folder="offload", offload_state_dict = True
)

Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

CPU times: user 39 s, sys: 26.1 s, total: 1min 5s
Wall time: 2min 25s


# Test the model

In [ ]:
prompt = '질문:\n' + '추진 목적은 ? ' + '\n\n답변:\n'

In [ ]:
prompt = "What is the output format of the tasks that the Florence-2 model can handle?"
#ground truth = "The output format of the tasks that the Florence-2 model can handle is text forms, whether it be captioning, object detection, grounding or segmentation."

In [ ]:
input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
outputs = new_model.generate(input_ids=input_ids,
                         max_new_tokens=100,
                        #  do_sample=True,
                        #  top_p=0.9,
                         temperature=0.2)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


RuntimeError: "LayerNormKernelImpl" not implemented for 'Half'

In [ ]:
result = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]

In [ ]:
print(result)

Question:
추진 목적은? 

Answer:
보험자가 재해를 원인으로 보험료를 징수하는 것
 재해가 풍수해에 한정되지 않고 태풍, 호우, 대설 등의 풍수해로 인한 피해를 입은 경우에도 보험료를 징수

- 풍수해보험은 자연재해로 인한 피해를 보상받기 위한 제도로, 보험가입자는 자연재해로 인한 피해를 입은 경우에 보험금을 지급받을 수 있음
- 다만, 풍수해보험은 보험가입금액이 1억 원을


In [ ]:
# Merge LoRA and base model
merged_model = new_model.merge_and_unload()

In [ ]:
# Save the merged model
merged_model.save_pretrained("metallama2-7b-qa-tuned-merged", safe_serialization=True)
tokenizer.save_pretrained("metallama2-7b-qa-tuned-merged")

('metallama2-7b-qa-tuned-merged/tokenizer_config.json',
 'metallama2-7b-qa-tuned-merged/special_tokens_map.json',
 'metallama2-7b-qa-tuned-merged/tokenizer.json')

In [ ]:
# !huggingface-cli login

In [ ]:
# push merged model to the hub
%%time
hf_model_repo = "genaitraining/llama-2-7b-qna-tuned"
merged_model.push_to_hub(hf_model_repo)
tokenizer.push_to_hub(hf_model_repo)

HfHubHTTPError: ignored

# Load the model from the HF Hub and test it

In [ ]:
import torch
from transformers import BitsAndBytesConfig

# Get the type
compute_dtype = getattr(torch, "float16")

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype
)

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

hf_model_repo = "genaitraining/llama-2-7b-qna-tuned"

# Get the tokenizer
tokenizer = AutoTokenizer.from_pretrained(hf_model_repo)

# Load the model
model = AutoModelForCausalLM.from_pretrained(hf_model_repo,
                                             quantization_config=bnb_config,
                                             device_map="auto")

In [ ]:
# prompt = "Question: What is the name of the new vision foundation model introduced in the paper?\n\nAnswer:\n"
# prompt = "Question: How does the Florence-2 model take user instructions?\n\nAnswer:\n"
# prompt = "Question: What is the output format of the tasks that the Florence-2 model can handle?\n\nAnswer:\n"
prompt = "Question: What is the main challenge addressed by the paper?\n\nAnswer:\n"

In [ ]:
# Generate response
%%time
input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids
outputs = model.generate(input_ids=input_ids,
                         max_new_tokens=200,
                         temperature=0.6)

result = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

# Print the result
print(f"Generated response:\n{result}")


# Use Transformers Pipeline for Inference

In [ ]:
import transformers

tokenizer = AutoTokenizer.from_pretrained("genaitraining/llama-2-7b-domain-tuned",  trust_remote_code=True)
pipeline = transformers.pipeline(
    "text-generation",
    model="genaitraining/llama-2-7b-domain-tuned",
    trust_remote_code=True

)

In [ ]:
%%time
sequences = pipeline(
    prompt,
    temperature=0.6,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
)

In [ ]:
for seq in sequences:
    print(seq['generated_text'])